# Projekt: Wykrywanie spamu wiadomości e-mail przy pomocy naiwnego klasyfikatora Bayesa i SVM

Autor: Konrad Woźniak

## Opis projektu

### Cel projektu

Celem projektu jest zastosowanie algorytmów wspomagania decyzji do procesu wspomagania filtracji wiadomości e-mail typu spam. Projekt opiera się na wykorzystaniu klasycznego zastosowania algorytmu Bayesowskiego do filtrowania spamu oraz algorytmu SVM. Jest to projekt wykonany w środowisku programistycznym Python, wykorzystując przy tym Jupyter Nootebook. 

<img src="stopspam.png" />

### Zbiór danych

Zbiór danych wykorzystany w projekcie został pobrany z serwisu kaggle.com i jest on plikiem typu csv. Plik posiada ponad 5700 różnych wiadomości.

Link do zbioru danych kaggle:

https://www.kaggle.com/datasets/studymart/spam-email-detection-dataset

### Naiwny Klasyfikator Bayesa

Jest to bardzo często stosowany algorytm w uczeniu maszynowym służący do odróżniania wiadomości typu spam od normalnych wiadomości.

Jego użycie polega na nauczeniu algorytmu ze zbioru treningowego wiadomości e-mail, które są oznaczone jako spam lub nie. Każda wiadomość jest opisana przez zbiór atrybutów, takich jak liczba wystąpień słów kluczowych, długość wiadomości, ilość załączników, czy adres nadawcy. Następnie, dla nowych wiadomości, naiwny klasyfikator Bayesowski oblicza prawdopodobieństwo, że dana wiadomość jest spamem lub nie-spamem, bazując na wartościach jej atrybutów. Jeśli prawdopodobieństwo, że wiadomość jest spamem, jest wyższe od pewnego ustalonego progu, to wiadomość jest uznawana za spam i trafia do folderu spamowego.

Przykładowo, jeśli w treningowym zbiorze wiadomości spamowych występuje wiele razy słowo "kredyt", a w niespamowych tylko kilka razy, to naiwny klasyfikator Bayesowski będzie skłaniał się do tego, że wiadomość zawierająca słowo "kredyt" jest spamem. W ten sposób, na podstawie analizy atrybutów wiadomości, algorytm jest w stanie dokonywać skutecznej klasyfikacji wiadomości jako spam lub nie-spam.

Każde prawdopodobieństwo pojedyńczego słowa, czy jest to wiadomość spamowa lub niespamowa jest określone poprzez:
ilość wszystkich użyć tego słowa podzielone przez wszystkie słowa użyte w wiadomościach jednej kategorii spamu lub nie.

Naiwnością klasyfikatora jest to, że niezależnie od szyku słów w wiadomości e-mailowej prawdopodobieństwo zawsze będzie takie samo, czyli kolejność w jakiej dane słowa zostały użyte nie ma znaczenia dla algorytmu.

## Implementacja projektu

### Import bibliotek projektowych

In [4]:
#Biblioteki
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
from nltk.corpus import stopwords
import nltk
import os
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn import model_selection
from sklearn import svm
from nltk import word_tokenize
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot
from sklearn.metrics import plot_confusion_matrix